In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [3]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object| Assignment Method | 
|----|-----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro|weighted|
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit|weighted|
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix|random|
|None|Car Heavy|NoWFH_Mix|commuter_model_nowfh_car|weighted|
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro|weighted|
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit|weighted|
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix|random|
|Medium|Car Heavy|NoWFH_Mix|commuter_model_midwfh_car|weighted|
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro|weighted|
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit|weighted|
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix|random|
|High|Car Heavy|NoWFH_Mix|commuter_model_highwfh_car|weighted|

In [4]:
commuter_model_nowfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_nowfh_transit = CommuterModel(ipums_df=ipums_df_19.copy()) 
commuter_model_nowfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_nowfh_car = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_midwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_car = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_highwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_car = CommuterModel(ipums_df=ipums_df_19.copy())

## No WFH

commuter_model_nowfh_micro  
commuter_model_nowfh_transit  
commuter_model_nowfh_mix  
commuter_model_nowfh_car  

In [5]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_nowfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_nowfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_transit.ipums_df['FLAG_WALK'] = commuter_model_nowfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_nowfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_nowfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

#### Transit
### Bus
commuter_model_nowfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_transit.bus_flag_binary(
                                        home_region=commuter_model_nowfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_nowfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_transit.subway_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_nowfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Zero Work From Home
commuter_model_nowfh_transit.ipums_df["FLAG_WFH"] = 0

# ### Assign transit mode hierarchically
# commuter_model_nowfh_transit.ipums_df['First_Assignment'] = commuter_model_nowfh_transit\
#             .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])
### Assign transit mode with weights -- transit heavy
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,0 #WFH
          )

commuter_model_nowfh_transit.ipums_df['First_Assignment'] = commuter_model_nowfh_transit.WeightedAssignNoOption(
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_nowfh_transit.ipums_df['TransMode'] = commuter_model_nowfh_transit.RandAssignChosenOption(
                                                                commuter_model_nowfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_nowfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_nowfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_nowfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_nowfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_micro.bus_flag_binary(
                                        home_region=commuter_model_nowfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_nowfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_micro.subway_flag_binary(
                                            home_region=commuter_model_nowfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_nowfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_nowfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )



## E-Scooter - bump up distance
commuter_model_nowfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_nowfh_micro.ipums_df['FLAG_WALK'] = commuter_model_nowfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_micro.ipums_df["FLAG_WFH"] = 0
# ### Assign transit mode hierarchically
# commuter_model_nowfh_micro.ipums_df['First_Assignment'] = commuter_model_nowfh_micro.AssignTransModesWaterfall(
#                                                     ["FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
#                                                                                 )
### Assign transit mode with weights -- micro heavy but with a little transit
WEIGHTS = (0 #Auto
,0 #Motorcycle
,0 #Taxi
,5 #EBus
,5 #Subway
,5 #CommuterRail
,5 #Ferry
,15 #EScooter
,40 #Walk
,25 #EBike
,0 #WFH
          )

commuter_model_nowfh_micro.ipums_df['First_Assignment'] = commuter_model_nowfh_micro.WeightedAssignNoOption(
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)


commuter_model_nowfh_micro.ipums_df['TransMode'] = commuter_model_nowfh_micro.RandAssignChosenOption(
                                                                commuter_model_nowfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_mix.bus_flag_binary(
                                        home_region=commuter_model_nowfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_mix.subway_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_mix.ipums_df['FLAG_WALK'] = commuter_model_nowfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_mix.ipums_df["FLAG_WFH"] = 0

## Keep random assignment for mixed transit option
commuter_model_nowfh_mix.ipums_df['First_Assignment'] = commuter_model_nowfh_mix.RandAssignNoOption(
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WFH'])
commuter_model_nowfh_mix.ipums_df['TransMode'] = commuter_model_nowfh_mix.RandAssignChosenOption(
                                                                commuter_model_nowfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [6]:
### ---------------------------------------------------------------------- ### 
## Car Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_nowfh_car.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_car.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 50
                                       ,female_pct = 50
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_nowfh_car.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_car.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_car.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_car.ipums_df['FLAG_WALK'] = commuter_model_nowfh_car.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=5
                                            ,female_pct=5
                                            )

## E-Bike
commuter_model_nowfh_car.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_car.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_car.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_nowfh_car.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_car.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_nowfh_car.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_car.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

#### Transit
### Bus
commuter_model_nowfh_car.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_car.bus_flag_binary(
                                        home_region=commuter_model_nowfh_car.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_car.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_car.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_nowfh_car.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_car.subway_flag_binary(
                                            home_region=commuter_model_nowfh_car.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_car.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_car.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_nowfh_car.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_car.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_car.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_car.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_car.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_car.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_car.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_car.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_car.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_car.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )
## Zero Work From Home
commuter_model_nowfh_car.ipums_df["FLAG_WFH"] = 0

### Assign transit mode with weights -- car heavy
WEIGHTS = (50 #Auto
           ,5 #Motorcycle
           ,5 #Taxi
           ,5 #EBus
           ,5 #Subway
           ,5 #CommuterRail
           ,5 #Ferry
           ,5 #EScooter
           ,5 #Walk
           ,5 #EBike
           ,0 #WFH
          )

commuter_model_nowfh_car.ipums_df['First_Assignment'] = commuter_model_nowfh_car.WeightedAssignNoOption(
                                                                commuter_model_nowfh_car.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_car.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_car.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_car.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_car.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_car.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_car.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_car.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_car.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_car.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_car.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_nowfh_car.ipums_df['TransMode'] = commuter_model_nowfh_car.RandAssignChosenOption(
                                                                commuter_model_nowfh_car.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_car.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [7]:
flag_cols = [x for x in commuter_model_nowfh_micro.ipums_df if "FLAG" in x]

In [8]:
commuter_model_nowfh_micro
commuter_model_nowfh_micro.ipums_df[flag_cols].sum()

FLAG_AUTO             1653
FLAG_MOTORCYCLE        204
FLAG_TAXICAB           286
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH                 0
dtype: int64

In [9]:
commuter_model_nowfh_transit
commuter_model_nowfh_transit.ipums_df[flag_cols].sum()

FLAG_AUTO             1692
FLAG_MOTORCYCLE        186
FLAG_TAXICAB           287
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           93
FLAG_WALK             1082
FLAG_EBIKE            7130
FLAG_WFH                 0
dtype: int64

In [10]:
commuter_model_nowfh_mix
commuter_model_nowfh_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1703
FLAG_MOTORCYCLE         37
FLAG_TAXICAB           235
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           82
FLAG_WALK             1082
FLAG_EBIKE            7179
FLAG_WFH                 0
dtype: int64

In [11]:
commuter_model_nowfh_car
commuter_model_nowfh_car.ipums_df[flag_cols].sum()

FLAG_AUTO             5467
FLAG_MOTORCYCLE        174
FLAG_TAXICAB           293
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER           85
FLAG_WALK               51
FLAG_EBIKE            7261
FLAG_WFH                 0
dtype: int64

## Medium WFH
commuter_model_midwfh_micro  
commuter_model_midwfh_transit  
commuter_model_midwfh_mix  
commuter_model_midwfh_car

In [12]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_midwfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_transit.ipums_df['FLAG_WALK'] = commuter_model_midwfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_midwfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_midwfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_midwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_midwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_midwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_midwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Medium Work From Home
commuter_model_midwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

# ### Assign transit mode hierarchically
# commuter_model_midwfh_transit.ipums_df['First_Assignment'] = commuter_model_midwfh_transit\
#             .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_WFH","FLAG_COMMUTERRAIL","FLAG_FERRY"])
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,40 #WFH
          )

commuter_model_midwfh_transit.ipums_df['First_Assignment'] = commuter_model_midwfh_transit.WeightedAssignNoOption(
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_midwfh_transit.ipums_df['TransMode'] = commuter_model_midwfh_transit.RandAssignChosenOption(
                                                                commuter_model_midwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [13]:
### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_midwfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_midwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_midwfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_midwfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_micro.bus_flag_binary(
                                        home_region=commuter_model_midwfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_midwfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_micro.subway_flag_binary(
                                            home_region=commuter_model_midwfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_midwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_midwfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )

## E-Scooter
commuter_model_midwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_midwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_midwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

                                                                                 
### Assign transit mode with weights -- micro heavy now with mid WFH
# but with a little transit
WEIGHTS = (0 #Auto
            ,0 #Motorcycle
            ,0 #Taxi
            ,5 #EBus
            ,5 #Subway
            ,5 #CommuterRail
            ,5 #Ferry
            ,15 #EScooter
            ,40 #Walk
            ,25 #EBike
            ,20 #WFH
          )

commuter_model_midwfh_micro.ipums_df['First_Assignment'] = commuter_model_midwfh_micro.WeightedAssignNoOption(
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_midwfh_micro.ipums_df['TransMode'] = commuter_model_midwfh_micro.RandAssignChosenOption(
                                                                commuter_model_midwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [14]:
### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_midwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_midwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_midwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

## Keep random assignment for mixed transit option
commuter_model_midwfh_mix.ipums_df['First_Assignment'] = commuter_model_midwfh_mix.RandAssignNoOption(
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_midwfh_mix.ipums_df['TransMode'] = commuter_model_midwfh_mix.RandAssignChosenOption(
                                                                commuter_model_midwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [15]:
### ---------------------------------------------------------------------- ### 
## Car Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_car.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_car.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 50
                                       ,female_pct = 50
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_midwfh_car.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_car.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_car.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_car.ipums_df['FLAG_WALK'] = commuter_model_midwfh_car.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=5
                                            ,female_pct=5
                                            )

## E-Bike
commuter_model_midwfh_car.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_car.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 15
                                        ,female_pct = 15
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_car.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_midwfh_car.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_car.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_midwfh_car.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_car.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_midwfh_car.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_car.bus_flag_binary(
                                        home_region=commuter_model_midwfh_car.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_car.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_car.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_midwfh_car.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_car.subway_flag_binary(
                                            home_region=commuter_model_midwfh_car.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_car.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_car.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_midwfh_car.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_car.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_car.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_car.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_car.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_car.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_car.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_car.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_car.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_car.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )
## Medium Work From Home
commuter_model_midwfh_car.ipums_df["FLAG_WFH"] = commuter_model_midwfh_car\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

# ### Assign transp. mode with weights
WEIGHTS = (50 #Auto
           ,5 #Motorcycle
           ,5 #Taxi
           ,5 #EBus
           ,5 #Subway
           ,5 #CommuterRail
           ,5 #Ferry
           ,5 #EScooter
           ,5 #Walk
           ,5 #EBike
           ,25 #WFH
          )

commuter_model_midwfh_car.ipums_df['First_Assignment'] = commuter_model_midwfh_car.WeightedAssignNoOption(
                                                                commuter_model_midwfh_car.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_car.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_car.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_car.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_car.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_car.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_car.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_car.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_car.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_car.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_car.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_midwfh_car.ipums_df['TransMode'] = commuter_model_midwfh_car.RandAssignChosenOption(
                                                                commuter_model_midwfh_car.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_car.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [16]:
commuter_model_midwfh_micro.ipums_df[flag_cols].sum() 

FLAG_AUTO             1639
FLAG_MOTORCYCLE        207
FLAG_TAXICAB           258
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              3297
dtype: int64

In [17]:
commuter_model_midwfh_transit.ipums_df[flag_cols].sum()  

FLAG_AUTO             1655
FLAG_MOTORCYCLE        196
FLAG_TAXICAB           274
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER          101
FLAG_WALK             1082
FLAG_EBIKE            7190
FLAG_WFH              3323
dtype: int64

In [18]:
commuter_model_midwfh_mix.ipums_df[flag_cols].sum()  

FLAG_AUTO             1696
FLAG_MOTORCYCLE         40
FLAG_TAXICAB           258
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           93
FLAG_WALK             1082
FLAG_EBIKE            7244
FLAG_WFH              3215
dtype: int64

In [19]:
commuter_model_midwfh_car.ipums_df[flag_cols].sum()  

FLAG_AUTO             5479
FLAG_MOTORCYCLE        194
FLAG_TAXICAB           285
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER           90
FLAG_WALK               50
FLAG_EBIKE            2416
FLAG_WFH              3302
dtype: int64

## High WFH
commuter_model_highwfh_micro  
commuter_model_highwfh_transit  
commuter_model_highwfh_mix    
commuter_model_highwfh_car

In [20]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_highwfh_transit.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_transit.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_highwfh_transit.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_transit.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_transit.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_transit.ipums_df['FLAG_WALK'] = commuter_model_highwfh_transit.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_transit.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_transit.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_transit.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_highwfh_transit.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_transit.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_highwfh_transit.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_transit.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_highwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_highwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_highwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_highwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## High Work From Home
commuter_model_highwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_highwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 1)

### Assign transit mode hierarchically
# commuter_model_highwfh_transit.ipums_df['First_Assignment'] = commuter_model_highwfh_transit\
#             .AssignTransModesWaterfall(["FLAG_WFH","FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,25 #EBus
           ,40 #Subway
           ,15 #CommuterRail
           ,10 #Ferry
           ,0 #EScooter
           ,5 #Walk
           ,0 #EBike
           ,100 #WFH
          )

commuter_model_highwfh_transit.ipums_df['First_Assignment'] = commuter_model_highwfh_transit.WeightedAssignNoOption(
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_transit.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_highwfh_transit.ipums_df['TransMode'] = commuter_model_highwfh_transit.RandAssignChosenOption(
                                                                commuter_model_highwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
## Auto
commuter_model_highwfh_micro.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_micro.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_highwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_micro.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_highwfh_micro.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_micro.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
## Bus
commuter_model_highwfh_micro.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_micro.bus_flag_binary(
                                        home_region=commuter_model_highwfh_micro.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_micro.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_micro.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )
## Subway
commuter_model_highwfh_micro.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_micro.subway_flag_binary(
                                            home_region=commuter_model_highwfh_micro.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_micro.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_micro.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Commuter Rail
commuter_model_highwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_micro.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_micro.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_micro.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_micro.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )
## Ferry
commuter_model_highwfh_micro.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_micro.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_micro.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_micro.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_micro.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )


## E-Scooter
commuter_model_highwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_highwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_highwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_highwfh_micro\
                                                    .wfh_flag_binary(wfh_dampener = 1)

# micro but with a little transit                                                              )
WEIGHTS = (0 #Auto
           ,0 #Motorcycle
           ,0 #Taxi
           ,5 #EBus
           ,5 #Subway
           ,5 #CommuterRail
           ,5 #Ferry
           ,15 #EScooter
           ,40 #Walk
           ,25 #EBike
           ,100 #WFH
          )

commuter_model_highwfh_micro.ipums_df['First_Assignment'] = commuter_model_highwfh_micro.WeightedAssignNoOption(
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_micro.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_highwfh_micro.ipums_df['TransMode'] = commuter_model_highwfh_micro.RandAssignChosenOption(
                                                                commuter_model_highwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_highwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_highwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_highwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 1)

## Keep random assignment for mixed transit option
commuter_model_highwfh_mix.ipums_df['First_Assignment'] = commuter_model_highwfh_mix.RandAssignNoOption(
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_highwfh_mix.ipums_df['TransMode'] = commuter_model_highwfh_mix.RandAssignChosenOption(
                                                                commuter_model_highwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")



### ---------------------------------------------------------------------- ### 
## Car Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_highwfh_car.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_car.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 5
                                       ,min_income = 32626
                                       ,male_pct = 50
                                       ,female_pct = 50
                                       ,age_dist = None
                                       )
## E-Scooter
commuter_model_highwfh_car.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_car.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_car.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_car.ipums_df['FLAG_WALK'] = commuter_model_highwfh_car.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=5
                                            ,female_pct=5
                                            )

## E-Bike
commuter_model_highwfh_car.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_car.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 15
                                        ,female_pct = 15
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_car.bike_friendly_origins
                                        )

## Motorcycle
commuter_model_highwfh_car.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_car.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 5
                                       ,female_pct = 1
                                       ,age_dist = None
                                        )
## Taxicab
commuter_model_highwfh_car.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_car.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
#### Transit
### Bus
commuter_model_highwfh_car.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_car.bus_flag_binary(
                                        home_region=commuter_model_highwfh_car.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_car.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_car.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_highwfh_car.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_car.subway_flag_binary(
                                            home_region=commuter_model_highwfh_car.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_car.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_car.time_Subway
                                            ,affordability=50
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_highwfh_car.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_car.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_car.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_car.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_car.time_CommuterRail
                                            ,affordability=50
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_car.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_car.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_car.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_car.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_car.time_Ferry
                                           ,affordability=50
                                           ,fixgaps=False
                                          )
## High Work From Home
commuter_model_highwfh_car.ipums_df["FLAG_WFH"] = commuter_model_highwfh_car\
                                                    .wfh_flag_binary(wfh_dampener = 1)

# ### Assign transp. mode with weights
WEIGHTS = (50 #Auto
           ,5 #Motorcycle
           ,5 #Taxi
           ,5 #EBus
           ,5 #Subway
           ,5 #CommuterRail
           ,5 #Ferry
           ,5 #EScooter
           ,5 #Walk
           ,5 #EBike
           ,100 #WFH
          )

commuter_model_highwfh_car.ipums_df['First_Assignment'] = commuter_model_highwfh_car.WeightedAssignNoOption(
                                                                commuter_model_highwfh_car.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_car.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_car.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_car.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_car.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_car.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_car.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_car.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_car.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_car.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_car.ipums_df['FLAG_WFH'],
                                                                WEIGHTS=WEIGHTS)

commuter_model_highwfh_car.ipums_df['TransMode'] = commuter_model_highwfh_car.RandAssignChosenOption(
                                                                commuter_model_highwfh_car.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_car.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [21]:
commuter_model_highwfh_micro.ipums_df[flag_cols].sum()  

FLAG_AUTO             1710
FLAG_MOTORCYCLE        210
FLAG_TAXICAB           263
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              6625
dtype: int64

In [22]:
commuter_model_highwfh_transit.ipums_df[flag_cols].sum()    

FLAG_AUTO             1603
FLAG_MOTORCYCLE        187
FLAG_TAXICAB           259
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           81
FLAG_WALK             1082
FLAG_EBIKE            7205
FLAG_WFH              6552
dtype: int64

In [23]:
commuter_model_highwfh_mix.ipums_df[flag_cols].sum()    

FLAG_AUTO             1705
FLAG_MOTORCYCLE         36
FLAG_TAXICAB           283
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           95
FLAG_WALK             1082
FLAG_EBIKE            7160
FLAG_WFH              6595
dtype: int64

In [24]:
commuter_model_highwfh_car.ipums_df[flag_cols].sum()    

FLAG_AUTO             5509
FLAG_MOTORCYCLE        176
FLAG_TAXICAB           278
FLAG_EBUSES          13866
FLAG_SUBWAY          16896
FLAG_COMMUTERRAIL     7459
FLAG_FERRY            2511
FLAG_ESCOOTER           93
FLAG_WALK               48
FLAG_EBIKE            2434
FLAG_WFH              6511
dtype: int64

In [25]:
### Compare
commuter_model_nowfh_micro.name = "commuter_model_nowfh_micro"
commuter_model_nowfh_transit.name = "commuter_model_nowfh_transit"
commuter_model_nowfh_mix.name = "commuter_model_nowfh_mix"
commuter_model_nowfh_car.name = "commuter_model_nowfh_car"

commuter_model_midwfh_micro.name = "commuter_model_midwfh_micro"
commuter_model_midwfh_transit.name = "commuter_model_midwfh_transit"
commuter_model_midwfh_mix.name = "commuter_model_midwfh_mix"
commuter_model_midwfh_car.name = "commuter_model_midwfh_car"

commuter_model_highwfh_micro.name = "commuter_model_highwfh_micro"
commuter_model_highwfh_transit.name = "commuter_model_highwfh_transit"
commuter_model_highwfh_mix.name = "commuter_model_highwfh_mix"
commuter_model_highwfh_car.name = "commuter_model_highwfh_car"

In [26]:
master_cts = pd.DataFrame()
col_names = list()
for model in [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
              ,commuter_model_nowfh_car
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
              ,commuter_model_midwfh_car
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix
             ,commuter_model_highwfh_car]:
    model_cts = model.ipums_df['First_Assignment'].value_counts(dropna=False)
    master_cts = pd.concat([master_cts,model_cts],axis=1)
    col_names.append(model.name)
master_cts.columns = col_names
master_cts

,commuter_model_nowfh_micro,commuter_model_nowfh_transit,commuter_model_nowfh_mix,commuter_model_nowfh_car,commuter_model_midwfh_micro,commuter_model_midwfh_transit,commuter_model_midwfh_mix,commuter_model_midwfh_car,commuter_model_highwfh_micro,commuter_model_highwfh_transit,commuter_model_highwfh_mix,commuter_model_highwfh_car
Bicycle,8464.0,NaN,2578.0,2375.0,8070.0,NaN,2487,748,6942.0,NaN,2396,635
CommuterRail,4178.0,3863.0,4273.0,2902.0,3555.0,3425.0,3835,2623,2590.0,2653.0,3576,2088
Bus,3156.0,5949.0,5646.0,4832.0,2988.0,5589.0,5451,4832,2547.0,5075.0,5189,4205
Subway,2978.0,11880.0,7984.0,7171.0,2833.0,11256.0,7677,7276,2425.0,9878.0,7347,6147
Escooter,2542.0,NaN,17.0,23.0,2429.0,NaN,23,30,2092.0,NaN,22,25
No Option,1578.0,1668.0,1439.0,1070.0,1369.0,1470.0,1248,999,1186.0,1278.0,1067,869
Ferry,587.0,518.0,924.0,766.0,526.0,463.0,897,719,462.0,368.0,861,639
Walk,495.0,100.0,335.0,19.0,486.0,81.0,301,13,442.0,73.0,339,17
AutoOccupants,NaN,NaN,705.0,4684.0,NaN,NaN,719,4541,NaN,NaN,648,3849
Taxicab,NaN,NaN,69.0,74.0,NaN,NaN,79,87,NaN,NaN,77,60


## Electric Model

### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object| Electric Model Object|
|----|-----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro|electric_model_nowfh_micro|
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit|electric_model_nowfh_transit|
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix|electric_model_nowfh_mix|
|None|Car|NoWFH_Car|commuter_model_nowfh_car|electric_model_nowfh_car|
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro|electric_model_midwfh_micro|
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit|electric_model_midwfh_transit|
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix|electric_model_midwfh_mix|
|Medium|Car|MidWFH_Car|commuter_model_midwfh_car|electric_model_midwfh_car|
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro|electric_model_highwfh_micro|
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit|electric_model_highwfh_transit|
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix|electric_model_highwfh_mix|
|High|Car|HighWFH_Car|commuter_model_highwfh_car|electric_model_highwfh_car|

In [27]:
electric_model_nowfh_micro = ElectricModel(commuter_model_nowfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_micro')

electric_model_nowfh_transit = ElectricModel(commuter_model_nowfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_transit')

electric_model_nowfh_mix = ElectricModel(commuter_model_nowfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_mix')

electric_model_nowfh_car = ElectricModel(commuter_model_nowfh_car.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_car')

# ---- # ---- # ---- # ---- # ---- # ---- #

electric_model_midwfh_micro = ElectricModel(commuter_model_midwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_micro')

electric_model_midwfh_transit = ElectricModel(commuter_model_midwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_transit')

electric_model_midwfh_mix = ElectricModel(commuter_model_midwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_mix')

electric_model_midwfh_car = ElectricModel(commuter_model_midwfh_car.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_car')

# ---- # ---- # ---- # ---- # ---- # ---- #


electric_model_highwfh_micro = ElectricModel(commuter_model_highwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_micro')

electric_model_highwfh_transit = ElectricModel(commuter_model_highwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_transit')

electric_model_highwfh_mix = ElectricModel(commuter_model_highwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_mix')

electric_model_highwfh_car = ElectricModel(commuter_model_highwfh_car.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_car')

In [28]:
ct= 0 
for e_m in [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_nowfh_car
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_midwfh_car
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix
    ,electric_model_highwfh_car]:
    ct+=1
    
    ## Run methods in order to read in, clean, and aggregate data
    e_m.read_electric_info_in()

    e_m.clean_commuter_df()

    e_m.create_in_out_flow_dfs()

    e_m.aggregate_data()
    print(f"electric model done for em {ct}: {e_m.model_name}")

ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 1: electric_model_nowfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 2: electric_model_nowfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 3: electric_model_nowfh_mix
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 4: electric_model_nowfh_car
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 5: electric_model_midwfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 6: electric_model_midwfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 7: electric_model_midwfh_mi

In [33]:
#### Late addendum - current scenario - no modeling. Just for baseline
commuter_model_2019transpmode = CommuterModel(ipums_df=ipums_df_19.copy())

### Create empty flag columns just for consistency:
for c in ["FLAG_AUTO","FLAG_MOTORCYCLE","FLAG_TAXICAB","FLAG_EBUSES","FLAG_SUBWAY"\
          ,"FLAG_COMMUTERRAIL","FLAG_FERRY","FLAG_ESCOOTER","FLAG_WALK","FLAG_EBIKE","FLAG_WFH"]:
    commuter_model_2019transpmode.ipums_df[c] = 0

### Assign first assignment and final assignment as current transportation mode
commuter_model_2019transpmode.ipums_df['First_Assignment'] = commuter_model_2019transpmode.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH']
commuter_model_2019transpmode.ipums_df['TransMode'] = commuter_model_2019transpmode.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH']
commuter_model_2019transpmode.name = "commuter_model_2019transpmode"

### Run electrical model
electric_model_2019transpmode = ElectricModel(commuter_model_2019transpmode.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_2019transpmode')

electric_model_2019transpmode.read_electric_info_in()

electric_model_2019transpmode.clean_commuter_df()

electric_model_2019transpmode.create_in_out_flow_dfs()

electric_model_2019transpmode.aggregate_data()



ev reference table read in. subway floor and commuter rail floor extracted from table


In [34]:
save_dirs = ["NoWFH_Micro"
,"NoWFH_Transit"
,"NoWFH_Mix"
,"NoWFH_Car"             
,"MidWFH_Micro"
,"MidWFH_Transit"
,"MidWFH_Mix"
,"MidWFH_Car"
,"HighWFH_Micro"
,"HighWFH_Transit"
,"HighWFH_Mix"
,"HighWFH_Car"
,"Baseline_2019_Mode"]

commuter_models = [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
              ,commuter_model_nowfh_car
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
              ,commuter_model_midwfh_car
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix
             ,commuter_model_highwfh_car
              ,commuter_model_2019transpmode]

electric_models = [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_nowfh_car
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_midwfh_car
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix
    ,electric_model_highwfh_car
    ,electric_model_2019transpmode]



In [37]:
save_dir = "../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3"
os.makedirs(save_dir,exist_ok=True)

for save_loc,commuter_model,electric_model in zip(save_dirs,commuter_models,electric_models):
    save_loc_models = f"{save_dir}/{save_loc}"
    print(save_loc_models)
    print(commuter_model.name," | ",electric_model.model_name)
    os.makedirs(save_loc_models,exist_ok=True)
    
    commuter_model.ipums_df.to_pickle(f"{save_loc_models}/commuter_model_ipums_df.pkl")
    
    electric_model.df_aggregate.to_pickle(f"{save_loc_models}/electric_model_df_aggregate.pkl")
    print()

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/NoWFH_Micro
commuter_model_nowfh_micro  |  electric_model_nowfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/NoWFH_Transit
commuter_model_nowfh_transit  |  electric_model_nowfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/NoWFH_Mix
commuter_model_nowfh_mix  |  electric_model_nowfh_mix

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/NoWFH_Car
commuter_model_nowfh_car  |  electric_model_nowfh_car

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/MidWFH_Micro
commuter_model_midwfh_micro  |  electric_model_midwfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/MidWFH_Transit
commuter_model_midwfh_transit  |  electric_model_midwfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos_V3/MidWFH_Mix
commuter_model_midwfh_mix  |  electric_model_m